# Oh dear!

If you've got here, then you're still having problems setting up your environment. I'm so sorry! Hang in there and we should have you up and running in no time.

Setting up a Data Science environment can be challenging because there's a lot going on under the hood. But we will get there.

And please remember - I'm standing by to help out. Message me or email ed@edwarddonner.com and I'll get on the case. The very last cell in this notebook has some diagnostics that will help me figure out what's happening.


# Step 1

Try running the next 2 cells (click in the cell under this one and hit shift+return, then shift+return again).

If this gives an error, then you're likely not running in an "activated" environment. Please check back in Part 5 of the SETUP guide for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) for setting up the Anaconda (or virtualenv) environment and activating it, before running `jupyter lab`.

If you look in the Anaconda prompt (PC) or the Terminal (Mac), you should see `(llms)` in your prompt where you launch `jupyter lab` - that's your clue that the llms environment is activated.

If you are in an activated environment, the next thing to try is to restart everything:
1. Close down all Jupyter windows, like this one
2. Exit all command prompts / Terminals / Anaconda
3. Repeat Part 5 from the SETUP instructions to begin a new activated environment and launch `jupyter lab` from the `llm_engineering` directory  
4. Come back to this notebook, and do Kernel menu >> Restart Kernel and Clear Outputs of All Cells
5. Try the cell below again.

If **that** doesn't work, then please contact me! I'll respond quickly, and we'll figure it out. Please run the diagnostics (last cell in this notebook) so I can debug. If you used Anaconda, it might be that for some reason your environment is corrupted, in which case the simplest fix is to use the virtualenv approach instead (Part 2B in the setup guides).

In [ ]:
# Some quick checks that your Conda environment or VirtualEnv is as expected
# The Environment Name should be: llms

import os

conda_prefix = os.environ.get('CONDA_PREFIX')
if conda_prefix:
    print("Anaconda environment is active:")
    print(f"Environment Path: {conda_prefix}")
    print(f"Environment Name: {os.path.basename(conda_prefix)}")

virtual_env = os.environ.get('VIRTUAL_ENV')
if virtual_env:
    print("Virtualenv is active:")
    print(f"Environment Path: {virtual_env}")
    print(f"Environment Name: {os.path.basename(virtual_env)}")

if not conda_prefix and not virtual_env:
    print("Neither Anaconda nor Virtualenv seems to be active. Did you start jupyter lab in an Activated environment? See Setup Part 5.")

In [ ]:
# And now, this should run with no output - no import errors.
# Import errors might indicate that you started jupyter lab without your environment activated? See SETUP part 5.
# Or you might need to restart your Kernel and Jupyter Lab.
# Or it's possible that something is wrong with Anaconda, in which case we may have to use virtualenv instead.
# If you're unsure, please run the diagnostics (last cell in this notebook) and then email me at ed@edwarddonner.com

from openai import OpenAI

# Step 2

Let's check your .env file exists and has the OpenAI key set properly inside it.  
Please run this code and check that it prints a successful message, otherwise follow its instructions.

Note that the `.env` file won't show up in your Jupyter Lab file browser, because Jupyter hides files that start with a dot for your security; they're considered hidden files. If you need to change the name, you'll need to use a command terminal or File Explorer (PC) / Finder Window (Mac). Ask ChatGPT if that's giving you problems, or email me!

If you're having challenges creating the `.env` file, we can also do it with code! See the cell after the next one.

It's important to launch `jupyter lab` from the project root directory, `llm_engineering`. If you didn't do that, this cell might give you problems.

In [ ]:
from pathlib import Path

parent_dir = Path("..")
env_path = parent_dir / ".env"

if env_path.exists() and env_path.is_file():
    print(".env file found.")

    # Read the contents of the .env file
    with env_path.open("r") as env_file:
        contents = env_file.readlines()

    key_exists = any(line.startswith("OPENAI_API_KEY=") for line in contents)
    good_key = any(line.startswith("OPENAI_API_KEY=sk-proj-") for line in contents)
    
    if key_exists and good_key:
        print("SUCCESS! OPENAI_API_KEY found and it has the right prefix")
    elif key_exists:
        print("Found an OPENAI_API_KEY although it didn't have the expected prefix sk-proj- \nPlease double check your key in the file..")
    else:
        print("Didn't find an OPENAI_API_KEY in the .env file")
else:
    print(".env file not found in the llm_engineering directory. It needs to have exactly the name: .env")
    
    possible_misnamed_files = list(parent_dir.glob("*.env*"))
    
    if possible_misnamed_files:
        print("\nWarning: No '.env' file found, but the following files were found in the llm_engineering directory that contain '.env' in the name. Perhaps this needs to be renamed?")
        for file in possible_misnamed_files:
            print(file.name)

## Fallback plan - python code to create the .env file for you

Only run the next cell if you're having problems making the .env file.  
Replace the text in the first line of code with your key from OpenAI.

In [ ]:
# Only run this code in this cell if you want to have a .env file created for you!

make_me_a_file_with_this_key = "put your key here inside these quotes.. it should start sk-proj-"

from pathlib import Path

parent_dir = Path("..")
env_path = parent_dir / ".env"

if env_path.exists():
    print("There is already a .env file - if you want me to create a new one, please delete the existing one first")
else:
    try:
        with env_path.open(mode='w', encoding='utf-8') as env_file:
            env_file.write(f"OPENAI_API_KEY={make_me_a_file_with_this_key}")
        print(f"Successfully created the .env file at {env_path}")
        print("Now rerun the previous cell to confirm that the file is created and the key is correct.")
    except Exception as e:
        print(f"An error occurred while creating the .env file: {e}")

# Step 3

Now let's check that your API key is correct set up in your `.env` file, and available using the dotenv package.
Try running the next cell.

In [ ]:
# This should print your API key to the output

import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found - please try Kernel menu >> Restart Kernel And Clear Outputs of All Cells")
elif api_key[:8]!="sk-proj-":
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

print("My key is", os.getenv("OPENAI_API_KEY"))

## It should print something like:

`My key is sk-proj-blahblahblah`

If it didn't print a key, then it's not able to find a file called `.env` in the `llm_engineering` folder.  
The name of the file must be exactly `.env` - it won't work if it's called `my-keys.env` or `.env.doc`.  
Double check those steps in the instructions. Is it possible that `.env` is actually called `.env.txt`? In Windows, you may need to change a setting in the File Explorer to ensure that file extensions are showing ("Show file extensions" set to "On"). You should also see file extensions if you type `dir` in the `llm_engineering` directory.

Nasty gotchas to watch out for:  
- In the .env file, there should be no space between the equals sign and the key. Like: `OPENAI_API_KEY=sk-proj-...`
- If you copied and pasted your API key from another application, make sure that it didn't replace hyphens in your key with long dashes  
- If you changed your .env file, you might need to restart your Jupyter "kernel" (the python process) to pick up the change via the Kernel menu >> Restart kernel, then rerun the cells from the top.

Worst case, if you're not able to get this part to work, it's not a big deal. You'll just have to paste your key into the Jupyter Notebook (see below for an example), and be sure to remove it before you share the Notebook with anybody else.

# Step 4

Now run the below code and you will hopefully see that GPT can handle basic arithmetic!!

If not, see the cell below.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# EITHER:
my_api_key = os.getenv("OPENAI_API_KEY")

# OR if you haven't been able to get .env working, uncomment this next line and paste your key inside the quote marks
# my_api_key = "REPLACE THIS TEXT WITH YOUR OPENAI API KEY WITHIN THE QUOTE MARKS - it should start sk-proj-"

print(f"Using API key {my_api_key}")

openai = OpenAI(api_key=my_api_key)
completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role":"user", "content": "What's 2+2?"}],
)
print(completion.choices[0].message.content)

# If the key was set correctly, and this still didn't work, perhaps with a RateLimit error

Then there's something up with your API key!

First check this webpage to make sure you have a positive credit balance.
OpenAI requires that you have a positive credit balance and it has minimums. My salespitch for OpenAI is that this is well worth it for your education: for less than the price of a music album, you will build so much valuable commercial experience. But it's not required for this course at all; you can watch me running OpenAI code, and then wait until we get to open source models in week 3. Also, I'll show you how to use Ollama to run open-source models locally.

https://platform.openai.com/settings/organization/billing/overview

Also try creating a new key (button on the top right) here:

https://platform.openai.com/api-keys

Sometimes OpenAI may take a few minutes to give you access after you try.

## If all else fails:

(1) Try pasting your error into ChatGPT or Claude! It's amazing how often they can figure things out

(2) Contact me! Please run the diagnostics in the cell below, then email me your problems to ed@edwarddonner.com

Thanks so much, and I'm sorry this is giving you bother!

# Gathering Essential Diagnostic information

## Please run this next cell to gather some important data

Please run the next cell; it should take a minute or so to run. Most of the time is checking your network bandwidth.
Then email me the output of the last cell to ed@edwarddonner.com.  
Alternatively: this will create a file called report.txt - just attach the file to your email.

In [ ]:
# Run my diagnostics report to collect key information for debugging
# Please email me the results. Either copy & paste the output, or attach the file report.txt

!pip install -q requests speedtest-cli psutil setuptools
from diagnostics import Diagnostics
Diagnostics().run()